# Final Project - Cloud Platforms - Section B Team 8

In this notebook, we predict a Chicago crashes dataset with the XGBoost algorithm. Before running this code, please make sure to create an S3 bucket and a DynamoDB with the primary key "modelId". Also be aware that you need to change the name of the S3 bucket and the DynamoDB in the code, and upload the crashes dataset to your S3 bucket. Finally, you need to give SageMaker the permissions to put objects into DynamoDB.  

## Environment preparation

This section prepares the environment needed for the modeling. 

### Necessary imports

We're importing boto3 and sagemaker because we plan to interact with AWS and use its machine learning services. For our data work, we're including numpy and pandas, which are essential for crunching numbers and handling data frames. Lastly, we've got train_test_split from sklearn, which is crucial for splitting our data into training and test sets, so we can validate our machine learning models effectively.

In [1]:
%pip install s3fs

  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
ERROR: Cannot uninstall fsspec 2023.6.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps fsspec==2023.6.0'.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri

import numpy as np
import io
import pandas as pd
from sklearn.model_selection import train_test_split

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### Load data & data cleaning

Read the data **from a S3 bucket to a CSV**. 

What we've done here is we've pulled the crash data from an S3 bucket, which is conveniently located in the US West (N. California) region—this choice gives us access to additional features that might be useful for our analysis. We’ve read in the specific columns we need from the CSV file, using pandas. To keep things clean, we’ve converted all column names to lowercase and created a binary 'injuries' column to easily identify records with injuries. We've also transformed the 'date' column into a datetime object, which lets us extract and analyze the data by month, weekday, and hour. After cleaning the data and dropping any rows with missing values, we've applied a filter to only include records with valid latitude readings. This cleaned dataset should give us a solid foundation for our analysis, and it’s all set for any modeling or deeper analysis we want to perform next.

In [3]:
data = pd.read_csv(
    's3://team-b8/crashes_2022.csv',
    usecols=['INJURIES_TOTAL', 'DATE', 'POSTED_SPEED_LIMIT', 'FIRST_CRASH_TYPE', 'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'TRAFFICWAY_TYPE', 'PRIM_CONTRIBUTORY_CAUSE', 'LATITUDE', 'LONGITUDE']
)

# Change column names to lower case
data.columns = data.columns.str.lower()

# Create new column called 'injuries' as a prediction target
data['injuries'] = np.where(data['injuries_total'] > 0, 1, 0)

# Ensure 'date' column is in datetime format for date/time extraction
data['date'] = pd.to_datetime(data['date'])

# Create new columns extracting date/time from 'date' column to analyze data on a more granular level
data['crash_month'] = data['date'].dt.month_name()
data['crash_weekday'] = data['date'].dt.day_name()
data['crash_hour'] = data['date'].dt.hour

# Create clean dataset with necessary values
crashes_clean = data.drop(['injuries_total', 'date'], axis=1)

# Drop rows with any missing values in the selected columns
crashes_clean = crashes_clean.dropna()

# Additionally, filter rows where latitude > 0
crashes_clean = crashes_clean[crashes_clean['latitude'] > 0]

crashes_clean.head()

,posted_speed_limit,first_crash_type,weather_condition,lighting_condition,trafficway_type,roadway_surface_cond,prim_contributory_cause,latitude,longitude,injuries,crash_month,crash_weekday,crash_hour
0,30,PEDESTRIAN,CLEAR,DAYLIGHT,NOT DIVIDED,DRY,UNABLE TO DETERMINE,41.691832,-87.671307,1,November,Wednesday,13
1,25,REAR TO SIDE,CLEAR,"DARKNESS, LIGHTED ROAD",FOUR WAY,DRY,FAILING TO YIELD RIGHT-OF-WAY,41.805552,-87.622887,1,November,Thursday,23
2,30,REAR END,CLEAR,DAYLIGHT,NOT DIVIDED,DRY,FAILING TO REDUCE SPEED TO AVOID CRASH,41.808378,-87.684571,0,June,Sunday,13
3,30,FIXED OBJECT,CLEAR,DAYLIGHT,DIVIDED - W/MEDIAN BARRIER,DRY,UNABLE TO DETERMINE,41.765892,-87.594228,0,May,Tuesday,13
4,35,REAR END,UNKNOWN,"DARKNESS, LIGHTED ROAD",DIVIDED - W/MEDIAN (NOT RAISED),UNKNOWN,FAILING TO REDUCE SPEED TO AVOID CRASH,41.692541,-87.618320,0,December,Tuesday,22


In [4]:
crashes_clean.shape

(107422, 13)

### Data preprocessing

We've set up a preprocessing pipeline to get our data ready for machine learning models. First, we separated our target variable, which is whether there were injuries, from the predictors. For numerical data, we created a pipeline to fill any missing values with the mean and then scale the values to have a standard distribution, which helps many models perform better. For the categorical data, we're tackling missing values by substituting them with the most frequent occurrence and then applying one-hot encoding to turn them into a format that our algorithms can work with. We've identified which of our columns are categorical and which are numerical, and we've used a ColumnTransformer to apply the appropriate transformations to each data type. This way, our dataset will be clean, preprocessed, and ready for the next steps.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


# Store prediction target in separate variable
X = crashes_clean.drop('injuries', axis=1)
y = crashes_clean['injuries']

# Define the preprocessing for numerical columns
# SimpleImputer to fill missing values with the mean, then scaling the data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define the preprocessing for categorical columns
# Filling missing values with the most frequent value then applying one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define categorical and numerical columns
categorical_cols = [
    'first_crash_type', 'weather_condition', 'lighting_condition',
    'trafficway_type', 'roadway_surface_cond', 'prim_contributory_cause',
    'crash_month', 'crash_weekday'
]

numerical_cols = ['posted_speed_limit', 'latitude', 'longitude', 'crash_hour']

# Create the ColumnTransformer with both transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ], remainder='passthrough')

### Train / val / test split & data preprocessing

We've divided our dataset into separate training, validation, and test sets to ensure we can train our models effectively and evaluate their performance accurately. We allocated 80% of the data for training and then split the remaining 20% equally for validation and testing. This helps us to tune the model parameters using the validation set without touching the test set, which is reserved for the final performance check.

Once our data was split, we applied our preprocessing steps to each set. We fitted the preprocessor to the training data, which means it learned what transformations to apply from this data, such as the mean for imputing missing values and the scaling parameters. Then, we transformed the training set with these fitted parameters. The validation and test sets were only transformed, not fitted, to mimic the real-world scenario where the model is applied to new, unseen data. This process is crucial for avoiding data leakage and ensuring that our model's performance metrics are genuine and reliable.

In [6]:
# Split data into training, validation, and test sets
X_train, X_testval, y_train, y_testval = train_test_split(X, y, train_size=0.8, random_state=1200)
X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval, train_size=0.5, random_state=1200)

# Apply preprocessing to the training, validation, and test sets
# Fit and transform the training set
X_train_preprocessed = preprocessor.fit_transform(X_train)
# Transform the validation and test set
X_val_preprocessed = preprocessor.transform(X_val)
X_test_preprocessed = preprocessor.transform(X_test)

### Transform preprocessed data

We've reached the stage where we're stitching everything back together after preprocessing. Our numerical features kept their names, which keeps things straightforward for us. But for our categorical features, since they went through one-hot encoding, they got new names based on their unique values, so we fetched these new names from our preprocessor.

With all of our feature names in hand, we combined them to keep track of our columns post-transformation. Because our preprocessing resulted in sparse matrices—efficient for storage when dealing with a lot of zeros—we used hstack to horizontally stack our target variable, y, back with our transformed features, X. Now, our training, validation, and test sets are back in a complete form, with rows representing records and columns representing features, ready for the learning stage.

In [9]:
from scipy.sparse import hstack

# Get the feature names since they were removed through the preprocessing
# For the numerical features, the names stay the same
numerical_features = numerical_cols

# For the categorical features, get the new column names from the one-hot encoder
categorical_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_cols)

# Combine the feature names from both transformations
all_features = list(numerical_features) + list(categorical_features)

# Recombine X and y for each set, taking into account that X_*_preprocessed are sparse matrices
train_preprocessed = hstack([y_train.values.reshape(-1, 1), X_train_preprocessed])
val_preprocessed = hstack([y_val.values.reshape(-1, 1), X_val_preprocessed])
test_preprocessed = hstack([y_test.values.reshape(-1, 1), X_test_preprocessed])

### View preprocessed data

To take a closer look and make it more human-readable, we're converting it into Compressed Sparse Row (CSR) format. This format is particularly good for quick row operations, which is exactly what we need.

Next, we're grabbing the first five rows of this CSR matrix and converting them to a dense format, meaning we're turning it back into a regular array with all the values filled in. This makes it possible to actually see the data instead of just the structure and non-zero entries.

In [10]:
from scipy.sparse import csr_matrix
# Convert matrix to CSR first for efficient row slicing
train_preprocessed_csr = csr_matrix(train_preprocessed)

# Slice the first 5 rows and convert them to dense format
train_preprocessed_head_dense = train_preprocessed_csr[:5].toarray()

# Create a DataFrame with the proper column names for the head of the dataset
train_preprocessed_head_df = pd.DataFrame(train_preprocessed_head_dense, columns=['injuries'] + all_features)

# Display the head of the DataFrame
train_preprocessed_head_df

,injuries,posted_speed_limit,latitude,longitude,crash_hour,first_crash_type_ANGLE,first_crash_type_ANIMAL,first_crash_type_FIXED OBJECT,first_crash_type_HEAD ON,first_crash_type_OTHER NONCOLLISION,...,crash_month_November,crash_month_October,crash_month_September,crash_weekday_Friday,crash_weekday_Monday,crash_weekday_Saturday,crash_weekday_Sunday,crash_weekday_Thursday,crash_weekday_Tuesday,crash_weekday_Wednesday
0,0.0,0.252922,0.627758,0.408668,-0.251554,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,-0.630523,-1.030480,0.552135,-0.982718,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.252922,0.015424,-0.960258,-0.068763,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.136367,-1.638799,0.610990,1.210774,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.252922,0.200728,0.925505,0.662401,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Define S3 upload function

We've crafted a function to handle the upload of our processed data back to S3. This function, upload_to_s3, first transforms the sparse matrix into a dense array, then into a pandas DataFrame, which we then write to a CSV format without any headers or index. We use an in-memory buffer, io.StringIO, for this, so we're not writing files to disk—neat and fast. After converting the data into a CSV string within the buffer, we simply upload this string to the specified S3 bucket and file path. By doing this, we're making sure that our preprocessed training and validation data are safely stored in the cloud, which is especially useful if we want to access this data again later or share it with others for further analysis or replication of our work.

In [11]:
s3 = boto3.resource('s3')

def upload_to_s3(matrix, bucket, filename):
    # Convert the sparse matrix to a dense format (numpy array), then to a DataFrame
    df = pd.DataFrame(matrix.toarray())
    placeholder = io.StringIO()
    df.to_csv(placeholder, header=False, index=False)
    # Upload csv string to S3
    object = s3.Object(bucket, filename)
    object.put(Body=placeholder.getvalue())

After defining this, the train and validation split are uploaded to the S3 bucket. 

In [12]:
upload_to_s3(train_preprocessed, 'team-b8', 'sagemaker-data/kc/train.csv')
upload_to_s3(val_preprocessed, 'team-b8', 'sagemaker-data/kc/val.csv')

## Setting up the model

We're utilizing the `Estimator` class from the `sagemaker.estimator` module, which essentially sets up a virtual environment for us to train our models on AWS without fussing over the details. Think of it like renting a lab equipped with all the gadgets we need for our experiments.

As for the model evaluation parameters, they're our yardsticks for performance. The AUC, or Area Under the Curve, tells us about the model's ability to classify between our classes—injuries or no injuries. A perfect model would have an AUC of 1, but we'll likely see a number less than that.

Logloss, or Logarithmic Loss, measures the accuracy of our classifier by penalizing false classifications. Lower logloss values mean better predictions, so we're aiming for as low as possible.

Lastly, we have the Error, which is simply the proportion of times our model predicts incorrectly. It's one minus the accuracy, so if our model was right 80% of the time, the error would be 20%. We're tracking these metrics to refine our model until it's as accurate as it can be.

We’ve configured the SageMaker Estimator with the necessary settings to train our model. This involves setting up an environment with XGBoost, specifying where to save our outputs in S3, and what metrics to use for evaluating model performance. We've then prepared the data channels, telling SageMaker where to find our training and validation data in S3, all formatted as CSV for convenience. With this in place, our model is ready to start learning from our data.

In [13]:
role = sagemaker.get_execution_role()
region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve('xgboost', region_name, version='0.90-1')
output_location = 's3://team-b8/sagemaker-output/'

hyperparams = {
    'eval_metric': 'auc,logloss,error',
    'num_round': '20',
    'objective': 'binary:logistic'
}

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=output_location,
    hyperparameters=hyperparams,
    sagemaker_session=sagemaker.Session()
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Now we create "channels". We need to specify where is the data and in which format in a specific dictionary:  

In [14]:
train_channel = sagemaker.session.s3_input(
    's3://team-b8/sagemaker-data/kc/train.csv',
    content_type='text/csv'
)
val_channel = sagemaker.session.s3_input(
    's3://team-b8/sagemaker-data/kc/val.csv',
    content_type='text/csv'
)

channels_for_training = {
    'train': train_channel,
    'validation': val_channel
}

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train first model (XGBoost)

We've just kicked off the training of our first model using XGBoost. With the estimator.fit command, SageMaker starts the training job, pulling in the data from the channels we set up. The logs are turned off for a cleaner output, but we can see from the info message that our training job is up and running. It goes through steps like starting up, preparing the instances, downloading the data, and finally running the training process. Once it's done training, it uploads the model artifacts back to S3. This sequence of steps is all standard for model training in SageMaker, and it's nice to see it all executing smoothly.


In [15]:
estimator.fit(inputs=channels_for_training, logs=False)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-04-02-17-27-04-355



2024-04-02 17:27:05 Starting - Starting the training job..
2024-04-02 17:27:20 Starting - Preparing the instances for training..........
2024-04-02 17:28:17 Downloading - Downloading input data.......
2024-04-02 17:28:57 Downloading - Downloading the training image....
2024-04-02 17:29:22 Training - Training image download completed. Training in progress.....
2024-04-02 17:29:47 Uploading - Uploading generated training model..
2024-04-02 17:30:04 Completed - Training job completed


### Retrieve metrics of the first model

In [16]:
# Get metrics of initial model
metrics = sagemaker.analytics.TrainingJobAnalytics(
    estimator._current_job_name,
    metric_names=['train:auc', 'validation:auc', 'train:logloss', 'validation:logloss', 'train:error', 'validation:error']
)
metrics.dataframe()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


,timestamp,metric_name,value
0,0.0,train:auc,0.786952
1,0.0,validation:auc,0.769895
2,0.0,train:logloss,0.390760
3,0.0,validation:logloss,0.393552
4,0.0,train:error,0.122934
5,0.0,validation:error,0.121160


### Interpretation of metrics

After our model finished training, we pulled the performance metrics to see how well it did. The training AUC of about 0.78 suggests our model is doing a pretty good job at distinguishing between classes—a higher score would be even better, but this is a solid start. The validation AUC is close to the training AUC, which is good because it means our model isn’t just memorizing the training data; it’s generalizing well to unseen data.

The log loss for both training and validation is around 0.39. Since log loss measures prediction error, a lower number is better, and this shows our model's predictions are reasonably well-calibrated.

Lastly, the error rate sits at around 0.12 for both training and validation, implying an accuracy of about 88%. This is a strong indicator that our model is making the right calls most of the time when predicting whether there were injuries in the crashes.

## Model tuning

We're fine-tuning our model's hyperparameters using SageMaker's tuning job. We've defined ranges for each parameter and set our model to focus on improving the AUC on the validation set. With early stopping after 10 rounds to prevent overfitting, the job will test up to 30 different settings to find the best ones. This systematic search is all about boosting the model's ability to predict accurately.

In [17]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.inputs import TrainingInput

# Specify the hyperparameter ranges
hyperparameter_ranges = {
    'eta': ContinuousParameter(0.01, 0.2),
    'min_child_weight': ContinuousParameter(1, 10),
    'alpha': ContinuousParameter(0, 2),
    'max_depth': IntegerParameter(3, 10),
    'subsample': ContinuousParameter(0.5, 1),
    'colsample_bytree': ContinuousParameter(0.5, 1),
    'gamma': ContinuousParameter(0, 5),
    'lambda': ContinuousParameter(1e-5, 10),
}

# Modify estimator
estimator.set_hyperparameters(
    eval_metric='auc,logloss,error',
    num_round=1000,  # Set a large number for num_round and rely on early stopping
    objective='binary:logistic',
    early_stopping_rounds=10
)

# Save tuning jobs in different path
estimator.output_path = 's3://team-b8/sagemaker-output/hyperparameter-tuning/'

# Specify the objective metric
objective_metric = 'validation:auc'

# Create and launch a hyperparameter tuning job
tuner = HyperparameterTuner(estimator,
                            objective_metric,
                            hyperparameter_ranges,
                            max_jobs=30,
                            max_parallel_jobs=10)

# Specify the training and validation data locations and content type
train_input = TrainingInput('s3://team-b8/sagemaker-data/kc/train.csv', content_type='text/csv')
validation_input = TrainingInput('s3://team-b8/sagemaker-data/kc/val.csv', content_type='text/csv')

# Launch a hyperparameter tuning job
tuner.fit({'train': train_input, 'validation': validation_input}, logs=False)

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-240402-1741


...................................................................................................!


### Retrieve metrics of model tuning

We've used SageMaker's session to retrieve the details of our best training job post-tuning. It's like we've held a contest for our models, and now we're meeting the winner. We fetched the winning hyperparameters and printed them out to see what made this model stand out. With these optimal settings in hand, we looked at the key performance metrics: AUC, log loss, and error on the validation set. Printing these metrics gave us insight into just how well this model is expected to perform when making predictions. It's a moment of truth, seeing if our tuning efforts have paid off in creating a more accurate and reliable model.

In [18]:
sagemaker_session = sagemaker.Session()

# Get the name of the best training job from hyperparameter tuning
best_training_job_name = tuner.best_training_job()

# Now get the details of the best training job
best_training_job_info = sagemaker_session.describe_training_job(best_training_job_name)

# The hyperparameters of the best training job are in the 'HyperParameters' key
best_hyperparameters = best_training_job_info['HyperParameters']

print(f"The best hyperparameters are: \n {best_hyperparameters}")

# Describe the best training job to get the metrics
metrics = best_training_job_info['FinalMetricDataList']

print("\nThe following metrics are from the best model:")
# Print out the metrics for the best training job
for metric in metrics:
    if metric['MetricName'] in ['validation:auc', 'validation:logloss', 'validation:error']:
        print(f"{metric['MetricName']}: {metric['Value']}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
The best hyperparameters are: 
 {'_tuning_objective_metric': 'validation:auc', 'alpha': '1.4071495043371078', 'colsample_bytree': '0.5436108896616062', 'early_stopping_rounds': '10', 'eta': '0.1059320330255047', 'eval_metric': 'auc,logloss,error', 'gamma': '1.7373337310860146', 'lambda': '0.03994046436954013', 'max_depth': '6', 'min_child_weight': '1.0', 'num_round': '1000', 'objective': 'binary:logistic', 'subsample': '0.6203075396151405'}

The following metrics are from the best model:
validation:logloss: 0.32312801480293274
validation:auc: 0.7994499802589417
validation:error: 0.11878599971532822


### Interpretation of metrics

The metrics from the best model after hyperparameter tuning tell us a promising story. We have a validation AUC of almost 0.80, indicating strong predictive power and the model's good ability to distinguish between the positive and negative classes. This is a key metric for classification problems, and being close to 1 is what we aim for.

The validation log loss at around 0.32 suggests our predictions are quite confident and generally accurate, as this metric punishes both confident wrong predictions and unconfident right ones.

A validation error of roughly 0.118—or about 11.8%—translates to an accuracy of approximately 88.2%. This means that our model correctly predicts whether there were injuries in a crash 88.2% of the time, which is quite effective for many practical applications.

The chosen hyperparameters that led to these metrics show a particular configuration of the model complexity, regularization, and the learning process. For instance, a 'max_depth' of 6 and 'min_child_weight' of approximately 1.0 suggest a certain level of complexity that the model can capture, while regularization parameters like 'alpha', 'lambda', and 'gamma' help prevent overfitting. The 'eta' value shows a moderate learning rate, and 'subsample' indicates a portion of the data used to grow trees, preventing overfitting as well.

All in all, these results suggest a well-tuned model that balances bias and variance to make reliable predictions on unseen data.

### Trial of retrieving feature importance

In our machine learning process, we attempted to extract feature importance from our trained model using AWS SageMaker. Despite correctly accessing the model artifacts from the S3 bucket and employing the XGBoost library, the Jupyter notebook environment consistently crashed upon execution of the loading code.

We have taken several measures to address this, such as checking for memory issues and ensuring XGBoost compatibility, yet the problem persists. For the time being, this technical hiccup has halted our ability to directly obtain feature importance. We plan to investigate this as next steps and find a resolution.

Below is the code we used in this attempt, documented for transparency and future troubleshooting efforts.

In [18]:
# import tarfile
# import xgboost as xgb
# import os

# s3_client = boto3.client('s3')

# Extract the S3 path where the model artifacts are stored
# model_artifacts_s3_path = best_training_job_info['ModelArtifacts']['S3ModelArtifacts']
# path_parts = model_artifacts_s3_path.replace("s3://", "").split("/", 1)
# bucket_name = path_parts[0]
# object_key = path_parts[1]

# Fetch the model artifacts file from S3
# response = s3_client.get_object(Bucket=bucket_name, Key=object_key)

# Read the file content into memory
# file_content = response['Body'].read()

# Specify a path in the SageMaker local file system
# local_model_dir = "/home/ec2-user/SageMaker/model-artifacts"
# os.makedirs(local_model_dir, exist_ok=True)

# Load the tar.gz file into a TarFile object
# model_file_name = 'xgboost-model'
# with tarfile.open(fileobj=io.BytesIO(file_content), mode="r:gz") as tar:
    # Try to extract the file with the known name 'xgboost-model'
    #try:
        # Extract the model file to the disk
        # tar.extract(model_file_name, path=local_model_dir)
    # except KeyError:
        # The file wasn't found in the archive, handle the exception as needed
        # print(f"The file {model_file_name} does not exist in the archive.")
        # raise
        
# model = xgb.Booster()
# model_path = os.path.join(local_model_dir, model_file_name)  # This is the file path to the model
# model.load_model(model_path)

# Extract feature importance
# feature_importance = model.get_score(importance_type='weight')
# print(feature_importance)

### Add information of best model to DynamoDB

In the following, the details of the best model (from hyperparameter tuning) are saved in a table inside the DynamoDB.

In [19]:
import json

# Initialize a DynamoDB client
dynamodb = boto3.client('dynamodb')

# Define the DynamoDB table name
table_name = 'BestModels_FinalAssignment'

desired_metrics = {}
desired_metric_names = ['validation:logloss', 'validation:auc', 'validation:error']
for metric in metrics:
    if metric['MetricName'] in desired_metric_names:
        desired_metrics[metric['MetricName']] = metric['Value']

# Convert hyperparameters and metrics to JSON strings
hyperparameters_json = json.dumps(best_hyperparameters)
metrics_json = json.dumps(desired_metrics)

# Insert the item into the DynamoDB table
response = dynamodb.put_item(
    TableName=table_name,
    Item={
        'modelId': {'S': best_training_job_name},
        'hyperparameters': {'S': hyperparameters_json},
        'metrics': {'S': metrics_json}
    }
)

## Deploying the model

Now that the model is ready, we can "deploy it". This will create an instance that "serves" the model continuosuly. This server will accept queries with input values in real time and will return the model prediction. 

In [20]:
# Create a new estimator object from the best training job
best_estimator = sagemaker.estimator.Estimator.attach(best_training_job_name)

# Now, deploy the best model
predictor = best_estimator.deploy(initial_instance_count=1,
                                  instance_type='ml.m4.xlarge',
                                  serializer=sagemaker.serializers.CSVSerializer())

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml

2024-04-02 17:47:32 Starting - Found matching resource for reuse
2024-04-02 17:47:32 Downloading - Downloading the training image
2024-04-02 17:47:32 Training - Training image download completed. Training in progress.
2024-04-02 17:47:32 Uploading - Uploading generated training model
2024-04-02 17:47:32 Completed - Resource released due to keep alive period expiry

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-04-02-18-23-50-567


INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-04-02-18-23-50-567
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-04-02-18-23-50-567


------!

### Predict data

We're on to predicting with our model, starting by carving out a small, manageable piece of the test data—the first five rows—to keep things simple. We've got a peek at the subset to ensure it looks correct before it hits the preprocessing pipeline, where it's scaled and encoded just like we did with the training data.

The preprocessed data is then converted from a sparse matrix to a dense format, necessary for making predictions. We put this into a pandas DataFrame, making it look like a neat table, and then output it to a CSV string, because that's what our model is trained to digest. Now, with our data neatly packaged, our model is ready to make some predictions.

In [21]:
# Create subset of X_test to predict
subset_X_test = X_test.iloc[:5]

# Display the subset of X_test with column names
print("Subset of X_test before preprocessing:")
print(subset_X_test)

# Preprocess the subset
subset_X_test_preprocessed = preprocessor.transform(subset_X_test)

# Convert the preprocessed subset to a dense format if it's sparse
subset_X_test_preprocessed_dense = subset_X_test_preprocessed.toarray()

# Create a DataFrame for the preprocessed data
subset_X_test_preprocessed_df = pd.DataFrame(subset_X_test_preprocessed_dense)

# Convert the preprocessed DataFrame to CSV format for prediction
placeholder = io.StringIO()
subset_X_test_preprocessed_df.to_csv(placeholder, header=False, index=False)
csv_data = placeholder.getvalue()

Subset of X_test before preprocessing:
       posted_speed_limit          first_crash_type weather_condition  \
67127                  30                  REAR END             CLEAR   
51878                  30      PARKED MOTOR VEHICLE              RAIN   
77924                  30  SIDESWIPE SAME DIRECTION             CLEAR   
36507                  30  SIDESWIPE SAME DIRECTION             CLEAR   
57305                  25                     ANGLE             CLEAR   

           lighting_condition                  trafficway_type  \
67127                DAYLIGHT                      NOT DIVIDED   
51878  DARKNESS, LIGHTED ROAD                      NOT DIVIDED   
77924                DAYLIGHT  DIVIDED - W/MEDIAN (NOT RAISED)   
36507                DAYLIGHT  DIVIDED - W/MEDIAN (NOT RAISED)   
57305                DAYLIGHT                         FOUR WAY   

      roadway_surface_cond      prim_contributory_cause   latitude  longitude  \
67127                  DRY        FOLLOWING 

In [22]:
# Predict data with deployed model endpoint
predictions = predictor.predict(csv_data)

predictions

b'0.08622758835554123,0.09884343296289444,0.03943060711026192,0.057139601558446884,0.2054167538881302'

### Interpretation of results

The output shows the predictions from our deployed model. Each number is the model's estimated probability of the positive class—in this case, the likelihood of injuries occurring in a crash. The values range from about 0.04 to 0.21, which suggests varying levels of risk across the different instances in our test subset. The closer the value is to 1, the higher the model assesses the risk of injury. For instance, the third prediction at approximately 0.04 indicates a lower probability of injuries compared to the last prediction, which is nearly 0.20. These results would help us identify instances with higher risks that may require further attention or intervention.

### Confusion matrix & generalization scores

We've put our model's predictions to the test, transforming the probabilities into binary outcomes to calculate a confusion matrix and key performance metrics. The accuracy tells us how often the model is right, precision and recall show us the quality of its positive predictions and its ability to capture the actual positives, respectively. The F1 score helps us balance precision and recall, and the ROC AUC score gauges the model’s ability to distinguish between classes. These metrics give us a well-rounded view of our model's effectiveness.

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Use preprocessed test set (without prediction target) from the beginning and convert sparse matrix to dense array
X_test_dense = X_test_preprocessed.toarray() 

# Create a DataFrame from the dense array
X_test_preprocessed_df = pd.DataFrame(X_test_dense)

# Convert to CSV format and predict
predictions = predictor.predict(X_test_preprocessed_df.to_csv(header=False, index=False)).decode('utf-8')
predicted_probabilities = np.fromstring(predictions, sep=',')
predicted_labels = (predicted_probabilities >= 0.5).astype(int)

# Calculate the confusion matrix with prediction target of test set
cm = confusion_matrix(y_test, predicted_labels)

cm_labeled = pd.DataFrame(cm, 
                     index=['Actual Negative', 'Actual Positive'], 
                     columns=['Predicted Negative', 'Predicted Positive'])

# Compute various performance metrics
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)
roc_auc = roc_auc_score(y_test, predicted_probabilities)  # Use predicted probabilities for ROC AUC

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

# Additionally, print the confusion matrix
print("Confusion Matrix:")
print(cm_labeled)

Accuracy: 0.8795
Precision: 0.7967
Recall: 0.2430
F1 Score: 0.3725
ROC AUC Score: 0.8059
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                9065                  98
Actual Positive                1196                 384


### Interpretation of results

These results paint a detailed picture of our model's performance. With an accuracy of 87.95%, our model is correctly predicting a large majority of the outcomes. However, the precision of 79.67% indicates that when the model predicts an injury, it's correct about 79% of the time. 

The recall, at 24.30%, is relatively low, suggesting that the model is only identifying about a quarter of all actual injury cases. This could imply that while our model is conservative in predicting injuries, it's missing quite a few that do occur.

The F1 score, which combines precision and recall into a single number, is 37.25%, reflecting that there's a significant imbalance between precision and recall. This could be due to a model that is very cautious about predicting an injury, thereby missing out on many true injury cases.

The ROC AUC score is 80.59%, a strong score that tells us the model is quite good at ranking predictions correctly, despite the low recall.

The confusion matrix gives us a breakdown of the actual predictions: 9065 true negatives and 384 true positives show correct predictions, while there are 98 false positives and 1196 false negatives. This confirms that our model is more likely to predict 'no injury' when in doubt, leading to a high number of false negatives and thus the low recall.

Overall, while the model is generally accurate, it is particularly cautious, leading to many missed injury cases. Depending on the application, this could be an area to improve, especially if the cost of missing true injury cases is high.